# Loading initial packages

In [3]:
import re
import numpy as np
import pandas as pd
import scipy
import sklearn
#from pandas.stats.api import ols
from subprocess import check_output
try:
    import statsmodels.formula.api as sm
except ImportError:
    import scikits.statsmodels.api as sm

# Reading in the data

In [4]:
TourneySeeds = pd.read_csv('data/TourneySeeds.csv')
SampleSubmission = pd.read_csv('data/SampleSubmission.csv')
Seasons = pd.read_csv('data/Seasons.csv')
Teams = pd.read_csv('data/Teams.csv')
TourneySlots = pd.read_csv('data/TourneySlots.csv')
TourneyDetailedResults = pd.read_csv('data/TourneyDetailedResults.csv')
TourneyCompactResults = pd.read_csv('data/TourneyCompactResults.csv')
team_dict = dict(zip(Teams['Team_Id'].values, Teams['Team_Name'].values))
RegularSeasonCompact = pd.read_csv('data/RegularSeasonCompactResults.csv')
TourneyDetailedResults['Wteam_name'] = TourneyDetailedResults['Wteam'].map(team_dict)
TourneyDetailedResults['Lteam_name'] = TourneyDetailedResults['Lteam'].map(team_dict)

# A quick look at the data

In [12]:
print(TourneySeeds.head(6))
print(TourneySlots.head(6))
print(SampleSubmission.head(6))
print(Seasons.head(6))
print(Teams.head(6))
print(TourneyDetailedResults.head(6))
print(TourneyCompactResults.head(6))

   Season Seed  Team
0    1985  W01  1207
1    1985  W02  1210
2    1985  W03  1228
3    1985  W04  1260
4    1985  W05  1374
5    1985  W06  1208
   Season  Slot Strongseed Weakseed
0    1985  R1W1        W01      W16
1    1985  R1W2        W02      W15
2    1985  R1W3        W03      W14
3    1985  R1W4        W04      W13
4    1985  R1W5        W05      W12
5    1985  R1W6        W06      W11
               Id  Pred
0  2016_1112_1114   0.5
1  2016_1112_1122   0.5
2  2016_1112_1124   0.5
3  2016_1112_1138   0.5
4  2016_1112_1139   0.5
5  2016_1112_1143   0.5
   Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988   11/2/1987    East    Midwest  Southeast       West
4    1989  10/31/1988    East       West    Midwest  Southeast
5    1990  10/30/1989    East    Midwest  Southeast    

# TrueSkill

## Add the package

In [17]:
import trueskill
from trueskill import Rating, global_env, rate_1vs1
env = trueskill.TrueSkill(draw_probability=0.44)
env.make_as_global()
print(global_env())

seasons = [2008,2009,2010,2011,2012,2013,2014,2015,2016]
print(seasons)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=44.0%)
[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]


## Data Cleaning
- Remove all games before 2008 (three point line was introduced)

In [39]:
# all games after 2008
RegPost2008 = RegularSeasonCompact[RegularSeasonCompact['Season'] >= 2008]
print(RegPost2008.head(6))
print(RegPost2008.tail(6))

SeasonGames = RegPost2008.loc[RegPost2008['Season'] == season,:]
print(SeasonGames.head(6))

       Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
97710    2008       0   1272     102   1404      71    H      0
97711    2008       0   1350      44   1263      42    N      0
97712    2008       1   1205      69   1105      55    N      0
97713    2008       1   1246      67   1146      40    H      0
97714    2008       1   1272      80   1350      63    H      0
97715    2008       1   1404      66   1263      62    N      0
        Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
145283    2016     131   1451      62   1285      59    N      0
145284    2016     132   1114      70   1419      50    N      0
145285    2016     132   1163      72   1272      58    N      0
145286    2016     132   1246      82   1401      77    N      1
145287    2016     132   1277      66   1345      62    N      0
145288    2016     132   1386      87   1433      74    N      0
       Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
97710    2008       0   1272     

## Apply TrueSkill to prior season data
- For each season, calculate the TrueSkill of each team in the season

In [43]:
# for each season
# calculate trueskill of all teams, then add it to the game log
# instantiate ratings for all teams and add it to the dataframe
output = pd.DataFrame()
for season in seasons:
    # get all the games in the season
    # print("Number of games in 2008:", len(RegPost2008[RegPost2008['Season'] == 2008]))
    print('assessing season ', season)
    SeasonGames = RegPost2008.loc[RegPost2008['Season'] == season,:]
    #print("sanity check: ", RegGames2008['Season'].count())
    Teams['Mu'] = Rating().mu
    Teams['Sigma'] = Rating().sigma
    print('initizlied teams =')
    print(Teams.head(6))
    
    for i, row in SeasonGames.iterrows():
        (team1, team2) = (row['Wteam'], row['Lteam'])
        #print('result:', row)
    
        # get index from team list
        iteam1 = Teams[Teams['Team_Id'] == team1].index[0]
        iteam2 = Teams[Teams['Team_Id'] == team2].index[0]
    
        # initialize rating object
        rating1 = Rating(Teams['Mu'][iteam1],Teams['Sigma'][iteam1])
        rating2 = Rating(Teams['Mu'][iteam2],Teams['Sigma'][iteam2])
        #print('old:',rating1.mu)
        #print('old:',rating2.mu)
    
        # rate them (unless it is a draw)
        if (row['Wscore'] == row['Lscore']):
            (rating1, rating2) = rate_1vs1(rating1, rating2, drawn=True)
        else:
            (rating1, rating2) = rate_1vs1(rating1, rating2)
    
        # update mu's and sigma's of the two teams
        Teams.loc[iteam1,'Mu'] = rating1.mu
        Teams.loc[iteam1,'Sigma'] = rating1.sigma
        Teams.loc[iteam2,'Mu'] = rating2.mu
        Teams.loc[iteam2,'Sigma'] = rating2.sigma
    
        #print('new1:', Teams.loc[iteam1,'Rating2008Mu'])
        #print('new2:', Teams.loc[iteam2,'Rating2008Mu'])
        #count = count+1
        #if count > 30:
        #    break

        
        #Teams.to_csv('2008trueskills.csv',index=False)
    print('season trueskills =')
    Teams.to_csv('trueskills-%i.csv' % season, index = False)
    print(Teams.head(6))
    
        
    # calculate margin of victory for each game
    SeasonGames['MOV'] = SeasonGames.Wscore-SeasonGames.Lscore
    #print(SeasonGames.head(6))


    # add trueskill ratings for each team
    Team1=pd.concat([Teams['Team_Id'], Teams['Mu'], Teams['Sigma']], axis=1, keys=['Wteam', 'Wm', 'Ws'])
    Team2=pd.concat([Teams['Team_Id'], Teams['Mu'], Teams['Sigma']], axis=1, keys=['Lteam', 'Lm', 'Ls'])
    SeasonGames = SeasonGames.merge(Team1, on='Wteam', how='left').merge(Team2, on='Lteam', how='left')
    print('game trueskills =')
    print (SeasonGames.head(6))
    
    output = output.append(SeasonGames, ignore_index=True)
    print('results')
    print(output.tail(6))
output.to_csv('output.csv',index=False)

assessing season  2008
initizlied teams =
   Team_Id    Team_Name  Mu     Sigma
0     1101  Abilene Chr  25  8.333333
1     1102    Air Force  25  8.333333
2     1103        Akron  25  8.333333
3     1104      Alabama  25  8.333333
4     1105  Alabama A&M  25  8.333333
5     1106   Alabama St  25  8.333333
season trueskills =
   Team_Id    Team_Name         Mu     Sigma
0     1101  Abilene Chr  25.000000  8.333333
1     1102    Air Force  27.255425  1.669827
2     1103        Akron  29.496638  1.547358
3     1104      Alabama  30.604314  1.493172
4     1105  Alabama A&M  15.322018  1.716729
5     1106   Alabama St  18.897214  1.699073
game trueskills =
   Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  MOV         Wm  \
0    2008       0   1272     102   1404      71    H      0   31  45.957676   
1    2008       0   1350      44   1263      42    N      0    2  29.093627   
2    2008       1   1205      69   1105      55    N      0   14  20.426175   
3    2008       1   124

C:\Users\hughsoong\Anaconda3\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


season trueskills =
   Team_Id    Team_Name         Mu     Sigma
0     1101  Abilene Chr  25.000000  8.333333
1     1102    Air Force  20.481142  1.932404
2     1103        Akron  27.354082  1.486150
3     1104      Alabama  31.263695  1.561389
4     1105  Alabama A&M   8.927384  1.717439
5     1106   Alabama St  20.963247  1.863906
game trueskills =
   Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  MOV         Wm  \
0    2009       7   1181      80   1342      49    H      0   31  43.662399   
1    2009       7   1204      65   1222      63    N      0    2  15.426299   
2    2009       8   1181      97   1204      54    H      0   43  43.662399   
3    2009       8   1222      76   1342      57    N      0   19  30.279356   
4    2009       8   1318      73   1237      60    N      0   13  27.294553   
5    2009       9   1275      70   1451      66    N      0    4  25.913385   

         Ws         Lm        Ls  
0  1.634098  19.862190  1.604237  
1  1.671206  30.279356 

## Linear Model: MOV ~ Wm + Ws + Lm + Ls
# this actually makes no sense

In [51]:
linmodel=sm.ols(formula = "MOV ~ Wm + Ws + Lm + Ls",data=output).fit()
print(linmodel.summary())
print(linmodel.mse_resid)
print(linmodel.mse_total)
#print(linmodel)

                            OLS Regression Results                            
Dep. Variable:                    MOV   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     3347.
Date:                Wed, 16 Mar 2016   Prob (F-statistic):               0.00
Time:                        01:26:19   Log-Likelihood:            -1.6692e+05
No. Observations:               47579   AIC:                         3.338e+05
Df Residuals:                   47574   BIC:                         3.339e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      9.4603      0.667     14.184      0.0

# This Season's Data
Now that we have a method and model, let's use the trueskill for this year's field to predict (margin of victory) win percentage between each possible matchup

In [64]:
# luckily, we already caluclated trueskill
TS2016 = pd.read_csv('trueskills-2016.csv')

# games to predict
game_to_predict = pd.concat([SampleSubmission['Id'],SampleSubmission['Id'].str.split('_', expand=True)], axis=1)
game_to_predict.rename(columns={0: 'season', 1: 'team1',2: 'team2'}, inplace=True)
game_to_predict['season'] = pd.to_numeric(game_to_predict['season'])
game_to_predict['team1'] = pd.to_numeric(game_to_predict['team1'])
game_to_predict['team2'] = pd.to_numeric(game_to_predict['team2'])
# add trueskill ratings for each team
Team1=pd.concat([TS2016['Team_Id'], TS2016['Team_Name'], TS2016['Mu'], TS2016['Sigma']], axis=1, keys=['team1', 'team1_name', 't1m', 't1s'])
Team2=pd.concat([TS2016['Team_Id'], TS2016['Team_Name'], TS2016['Mu'], TS2016['Sigma']], axis=1, keys=['team2', 'team2_name','t2m', 't2s'])
game_to_predict = game_to_predict.merge(Team1, on='team1', how='left').merge(Team2, on='team2', how='left')
print (game_to_predict.head(6))

#game_to_predict['predicted_mov'] = linmodel.predict(game_to_predict[['t1m','t1s',]])

from math import sqrt
from trueskill import BETA
from trueskill.backends import cdf

def win_probability(t1m,t1s,t2m,t2s):
    player_rating = Rating(t1m,t1s)
    opponent_rating = Rating(t2m,t2s)
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    res1 = cdf(delta_mu / denom)
    return res1 

game_to_predict['win_prob'] = np.nan
for i, row in game_to_predict.iterrows():
    game_to_predict.loc[i,'win_prob'] = win_probability(row['t1m'],row['t1s'],row['t2m'],row['t2s'])

print(game_to_predict.head(6))
game_to_predict.to_csv('predictions.csv',index=False)

               Id  season  team1  team2 team1_name        t1m       t1s  \
0  2016_1112_1114    2016   1112   1114    Arizona  38.609925  1.549138   
1  2016_1112_1122    2016   1112   1122    Arizona  38.609925  1.549138   
2  2016_1112_1124    2016   1112   1124    Arizona  38.609925  1.549138   
3  2016_1112_1138    2016   1112   1138    Arizona  38.609925  1.549138   
4  2016_1112_1139    2016   1112   1139    Arizona  38.609925  1.549138   
5  2016_1112_1143    2016   1112   1143    Arizona  38.609925  1.549138   

        team2_name        t2m       t2s  
0  Ark Little Rock  32.017114  1.933285  
1      Austin Peay  22.575793  1.444581  
2           Baylor  37.617276  1.556551  
3          Buffalo  28.556519  1.440394  
4           Butler  37.475693  1.652938  
5       California  37.796118  1.478383  
               Id  season  team1  team2 team1_name        t1m       t1s  \
0  2016_1112_1114    2016   1112   1114    Arizona  38.609925  1.549138   
1  2016_1112_1122    2016   11

In [ ]:
# add team names to data


# (EXAMPLE) Seed-Based model

## Extract seeds for each team

In [10]:
TourneySeeds['SeedNum'] = TourneySeeds['Seed'].apply(lambda x: re.sub("[A-z+a-z]","",x,flags=re.IGNORECASE))
print(TourneySeeds.tail(10))

      Season  Seed  Team SeedNum
2072    2016   Z08  1386      08
2073    2016   Z09  1153      09
2074    2016   Z10  1433      10
2075    2016   Z11  1320      11
2076    2016   Z12  1463      12
2077    2016   Z13  1423      13
2078    2016   Z14  1453      14
2079    2016   Z15  1167      15
2080    2016  Z16a  1221      16
2081    2016  Z16b  1380      16


In [11]:
game_to_predict = pd.concat([SampleSubmission['Id'],SampleSubmission['Id'].str.split('_', expand=True)], axis=1)
game_to_predict.rename(columns={0: 'season', 1: 'team1',2: 'team2'}, inplace=True)
game_to_predict['season'] = pd.to_numeric(game_to_predict['season'])
game_to_predict['team1'] = pd.to_numeric(game_to_predict['team1'])
game_to_predict['team2'] = pd.to_numeric(game_to_predict['team2'])
TourneySeeds['Season'] = pd.to_numeric(TourneySeeds['Season'])
TourneySeeds['Team'] = pd.to_numeric(TourneySeeds['Team'])
TourneySeeds['SeedNum'] = pd.to_numeric(TourneySeeds['SeedNum'])
game_to_predict = pd.merge(game_to_predict,TourneySeeds[['Season','Team','SeedNum']].rename(columns={'Season': 'season', 'Team': 'team1','SeedNum':'TeamSeed1'}),how='left',on=['season','team1'])
game_to_predict = pd.merge(game_to_predict,TourneySeeds[['Season','Team','SeedNum']].rename(columns={'Season': 'season', 'Team': 'team2','SeedNum':'TeamSeed2'}),how='left',on=['season','team2'])
print(game_to_predict.head(10))

               Id  season  team1  team2  TeamSeed1  TeamSeed2
0  2016_1112_1114    2016   1112   1114          6         12
1  2016_1112_1122    2016   1112   1122          6         16
2  2016_1112_1124    2016   1112   1124          6          5
3  2016_1112_1138    2016   1112   1138          6         14
4  2016_1112_1139    2016   1112   1139          6          9
5  2016_1112_1143    2016   1112   1143          6          4
6  2016_1112_1151    2016   1112   1151          6         12
7  2016_1112_1153    2016   1112   1153          6          9
8  2016_1112_1160    2016   1112   1160          6          8
9  2016_1112_1163    2016   1112   1163          6          9


## Join results to seeds

In [12]:
compact_results = pd.merge(TourneyCompactResults, TourneySeeds[['Season','Team','SeedNum']].rename(columns={'Team': 'Wteam','SeedNum':'WSeedNum'}), how='left', on=['Season','Wteam'])
compact_results = pd.merge(compact_results, TourneySeeds[['Season','Team','SeedNum']].rename(columns={'Team': 'Lteam','SeedNum':'LSeedNum'}), how='left', on=['Season','Lteam'])
print(compact_results.head(6))

   Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  WSeedNum  \
0    1985     136   1116      63   1234      54    N      0         9   
1    1985     136   1120      59   1345      58    N      0        11   
2    1985     136   1207      68   1250      43    N      0         1   
3    1985     136   1229      58   1425      55    N      0         9   
4    1985     136   1242      49   1325      38    N      0         3   
5    1985     136   1246      66   1449      58    N      0        12   

   LSeedNum  
0         8  
1         6  
2        16  
3         8  
4        14  
5         5  


## Fix wins

In [13]:
set1 = compact_results[['WSeedNum','LSeedNum']].rename(columns={'WSeedNum': 'Team1Seed','LSeedNum':'Team2Seed'})
set1['Team1Win'] = 1
set2 = compact_results[['LSeedNum','WSeedNum']].rename(columns={'LSeedNum': 'Team1Seed','WSeedNum':'Team2Seed'})
set2['Team1Win'] = 0
full_set = pd.concat([set1,set2],ignore_index=True)
full_set['Team1Seed'] = pd.to_numeric(full_set['Team1Seed'])
full_set['Team2Seed'] = pd.to_numeric(full_set['Team2Seed'])
full_set['Team1Win'] = pd.to_numeric(full_set['Team1Win'])

print(full_set.head(6))

   Team1Seed  Team2Seed  Team1Win
0          9          8         1
1         11          6         1
2          1         16         1
3          9          8         1
4          3         14         1
5         12          5         1


## Linear model

In [14]:
linmodel=ols(y=full_set['Team1Win'],x=full_set['Team2Seed']-full_set['Team1Seed'])
print(linmodel)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         3966
Number of Degrees of Freedom:   2

R-squared:         0.2525
Adj R-squared:     0.2523

Rmse:              0.4324

F-stat (1, 3964):  1339.2037, p-value:     0.0000

Degrees of Freedom: model 1, resid 3964

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x     0.0333     0.0009      36.60     0.0000     0.0315     0.0351
     intercept     0.5000     0.0069      72.82     0.0000     0.4865     0.5135
---------------------------------End of Summary---------------------------------



In [15]:
game_to_predict['Pred'] = linmodel.predict(x=game_to_predict['TeamSeed2']-game_to_predict['TeamSeed1'])
game_to_predict[['Id','Pred']].to_csv('seed_submission.csv',index=False)